In [1]:
from keras.models import Sequential
from keras.layers import Conv1D, GlobalMaxPooling1D, Embedding, LSTM
from keras.layers.core import Dense, Dropout, Activation
from keras.preprocessing.text import Tokenizer
from keras import metrics, regularizers
from keras.preprocessing import sequence
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

Using TensorFlow backend.


In [2]:
data = pd.read_csv('bio_clean.csv', header = 0, names = ['affliations', 'article_name'])

In [4]:
#Split the dataset to Training and Test subsets (90/10)
train, test = train_test_split(data, test_size = 0.1, random_state = 17) #random_state = None

train_descs = train['article_name']
train_labels = train['affliations']
 
test_descs = test['article_name']
test_labels = test['affliations']

In [5]:
# Model Parameters
vocab_size = 2000

sequences_length = 1200

embedding_dimensionality = 64 #possibly low??
max_features = 2000 #equal to vocab_size

num_labels = len(train_labels.unique())
batch_size = 32
nb_epoch = 20

nof_filters = 200 #check + research ... random now
kernel_size = 16 #check + research ... random now

hidden_dims = 1024

In [6]:
# Convert Texts to Numeric Vectors for Input
tokenizer = Tokenizer(num_words = vocab_size)
tokenizer.fit_on_texts(train_descs)

x_train = tokenizer.texts_to_sequences(train_descs)
x_test = tokenizer.texts_to_sequences(test_descs)

x_train = sequence.pad_sequences(x_train, maxlen = sequences_length, padding = 'post')
x_test = sequence.pad_sequences(x_test, maxlen = sequences_length, padding = 'post')

encoder = LabelBinarizer()
encoder.fit(train_labels)
y_train = encoder.transform(train_labels)
y_test = encoder.transform(test_labels)

In [7]:
model = Sequential()
model.add(Embedding(max_features, embedding_dimensionality, input_length = 1200))
#model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))

model.add(Conv1D(nof_filters, kernel_size, padding='valid', activation='relu', strides = 1))
model.add(GlobalMaxPooling1D())

model.add(Dense(hidden_dims))
model.add(Dropout(0.3))
model.add(Activation('relu'))

model.add(Dense(num_labels))
model.add(Activation('softmax'))

model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', #'sgd', 'adam', 'RMSprop', 'Adagrad'
                   metrics = [metrics.categorical_accuracy])

Instructions for updating:
Colocations handled automatically by placer.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1200, 64)          128000    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 1185, 200)         205000    
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              205824    
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
activation_1 (Activation)    (None, 1024)              0         
________________________________________________

In [8]:
history = model.fit(x_train, y_train,
                    batch_size = batch_size,
                    epochs = nb_epoch,
                    verbose = True,
                    validation_split = 0.2)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 12 samples, validate on 3 samples
Epoch 1/20
12/12 [==============================] - 3s 253ms/step - loss: 2.0722 - categorical_accuracy: 0.2500 - val_loss: 2.0532 - val_categorical_accuracy: 0.3333
Epoch 2/20
12/12 [==============================] - 1s 46ms/step - loss: 1.9767 - categorical_accuracy: 0.5833 - val_loss: 2.0239 - val_categorical_accuracy: 0.3333
Epoch 3/20
12/12 [==============================] - 1s 48ms/step - loss: 1.8691 - categorical_accuracy: 0.5833 - val_loss: 1.9983 - val_categorical_accuracy: 0.3333
Epoch 4/20
12/12 [==============================] - 1s 49ms/step - loss: 1.7367 - categorical_accuracy: 0.5833 - val_loss: 1.9993 - val_categorical_accuracy: 0.3333
Epoch 5/20
12/12 [==============================] - 1s 46ms/step - loss: 1.6032 - categorical_accuracy: 0.5833 - val_loss: 2.0753 - val_categorical_accuracy: 0.3333
Epoch

In [9]:
score = model.evaluate(x_test, y_test, batch_size = batch_size, verbose = True)
 
print('\nTest categorical_crossentropy:', score[0])
print('Categorical accuracy:', score[1])

2/2 [==============================] - 0s 10ms/step

Test categorical_crossentropy: 0.5607707500457764
Categorical accuracy: 1.0
